So uh, I recently learned the top right triangle is for theorized combinations. Oops, I have to do some data hackery to only scrape the bottom left triangle.

In [ ]:
// Import necessary packages
#r "nuget: FSharp.Data"

open FSharp.Data
open System.Reflection
open Microsoft.FSharp.Reflection


// Load the CSV file
type PigmentCsv = CsvProvider<"../data/raw/Pigments-2025-11-22.csv", HasHeaders=false>
let pigmentData = PigmentCsv.Load("../data/raw/Pigments-2025-11-22.csv")

// Define types
type Combo = {
    pigment1: string
    pigment2: string
    result: string
}

// Convert rows to matrix of strings
let pigmentMatrix : string[][] =
    pigmentData.Rows
    |> Seq.map (fun row ->
        let rowType = row.GetType()
        if FSharpType.IsTuple rowType then
            FSharpValue.GetTupleFields row
            |> Array.map (fun v -> v.ToString())
        else
            failwith "Row is not a tuple type"
    )
    |> Seq.toArray

// Define the row and column offsets
let offsetY = 1 // skip first 5 rows
let offsetX = 6 // skip first 5 columns

// Extract combos from lower triangle only (excluding diagonal)
let combos =
    [ for i in offsetY + 1 .. pigmentMatrix.Length - 1 do
        let pigment1 = pigmentMatrix[i][0] // pigment name from row i
        let maxJ = min (i + offsetX - 1) (pigmentMatrix[i].Length - 1)

        for j in offsetX .. maxJ do
            let result = pigmentMatrix[i][j]
            if not (System.String.IsNullOrWhiteSpace result) then
                let pigment2 = pigmentMatrix[0][j]
                yield {
                    pigment1 = pigment1
                    pigment2 = pigment2
                    result = result
                }
    ]

// Output result
combos



In [ ]:
open System.Text.Json
open System.Text.Json.Serialization
open System.IO

// Serialize combos to JSON
let jsonOptions = JsonSerializerOptions(WriteIndented = true)
let json = JsonSerializer.Serialize(combos, jsonOptions)

// Ensure the directory exists
let outputDir = "../data/extracted"
Directory.CreateDirectory(outputDir) |> ignore

// Write to file
let outputPath = Path.Combine(outputDir, "extracted.json")
File.WriteAllText(outputPath, json)

printfn "✅ Written %d combos to %s" combos.Length outputPath
